# Operaciones básicas con DataFrames

## Descripción de las variables

El dataset, obtenido de <a target = "_blank" href="https://www.transtats.bts.gov/Fields.asp?Table_ID=236">este link</a> está compuesto por las siguientes variables referidas siempre al año 2018:

1. **Month** 1-4
2. **DayofMonth** 1-31
3. **DayOfWeek** 1 (Monday) - 7 (Sunday)
4. **FlightDate** fecha del vuelo
5. **Origin** código IATA del aeropuerto de origen
6. **OriginCity** ciudad donde está el aeropuerto de origen
7. **Dest** código IATA del aeropuerto de destino
8. **DestCity** ciudad donde está el aeropuerto de destino  
9. **DepTime** hora real de salida (local, hhmm)
10. **DepDelay** retraso a la salida, en minutos
11. **ArrTime** hora real de llegada (local, hhmm)
12. **ArrDelay** retraso a la llegada, en minutos: se considera que un vuelo ha llegado "on time" si aterrizó menos de 15 minutos más tarde de la hora prevista en el Computerized Reservations Systems (CRS).
13. **Cancelled** si el vuelo fue cancelado (1 = sí, 0 = no)
14. **CancellationCode** razón de cancelación (A = aparato, B = tiempo atmosférico, C = NAS, D = seguridad)
15. **Diverted** si el vuelo ha sido desviado (1 = sí, 0 = no)
16. **ActualElapsedTime** tiempo real invertido en el vuelo
17. **AirTime** en minutos
18. **Distance** en millas
19. **CarrierDelay** en minutos: El retraso del transportista está bajo el control del transportista aéreo. Ejemplos de sucesos que pueden determinar el retraso del transportista son: limpieza de la aeronave, daño de la aeronave, espera de la llegada de los pasajeros o la tripulación de conexión, equipaje, impacto de un pájaro, carga de equipaje, servicio de comidas, computadora, equipo del transportista, problemas legales de la tripulación (descanso del piloto o acompañante) , daños por mercancías peligrosas, inspección de ingeniería, abastecimiento de combustible, pasajeros discapacitados, tripulación retrasada, servicio de inodoros, mantenimiento, ventas excesivas, servicio de agua potable, denegación de viaje a pasajeros en mal estado, proceso de embarque muy lento, equipaje de mano no válido, retrasos de peso y equilibrio.
20. **WeatherDelay** en minutos: causado por condiciones atmosféricas extremas o peligrosas, previstas o que se han manifestado antes del despegue, durante el viaje, o a la llegada.
21. **NASDelay** en minutos: retraso causado por el National Airspace System (NAS) por motivos como condiciones meteorológicas (perjudiciales pero no extremas), operaciones del aeropuerto, mucho tráfico aéreo, problemas con los controladores aéreos, etc.
22. **SecurityDelay** en minutos: causado por la evacuación de una terminal, re-embarque de un avión debido a brechas en la seguridad, fallos en dispositivos del control de seguridad, colas demasiado largas en el control de seguridad, etc.
23. **LateAircraftDelay** en minutos: debido al propio retraso del avión al llegar, problemas para conseguir aterrizar en un aeropuerto a una hora más tardía de la que estaba prevista.

Leemos el fichero CSV utilizando el delimitador por defecto de Spark (","). La primera línea contiene encabezados (nombres de columnas) por lo que no es parte de los datos y debemos indicarlo con la opción header.

In [3]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession\
    .builder\
    .appName("OPBasicas")\
    .getOrCreate()

In [4]:
# Esto no hace nada: la lectura es lazy así que no se lee en realidad hasta que ejecutemos una acción sobre flightsDF
# Solamente se comprueba que exista el fichero en esa ruta, y se leen los nombres de columnas
flightsDF = spark.read.option("header", "true")\
                 .csv("flights-jan-apr-2018.csv")

In [5]:
# Veamos el esquema (nombre y tipo de dato de cada columna). Esto son solamente metadatos, por lo que no es ninguna acción.
flightsDF.printSchema()

root
 |-- Month: string (nullable = true)
 |-- DayofMonth: string (nullable = true)
 |-- DayOfWeek: string (nullable = true)
 |-- FlightDate: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- OriginCity: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- DestCity: string (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- Cancelled: string (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- Distance: string (nullable = true)
 |-- CarrierDelay: string (nullable = true)
 |-- WeatherDelay: string (nullable = true)
 |-- NASDelay: string (nullable = true)
 |-- SecurityDelay: string (nullable = true)
 |-- LateAircraftDelay: string (nullable = true)



Todas las columnas son cadenas de caracteres porque no hemos indicado el tipo de dato para cada columna ni tampoco le hemos pedido a Spark que intente inferirlo a partir de los datos. No queremos que todas sean string porque hay algunas numéricas que deberían ser tratadas como tales. Vamos a intentar inferir el esquema. Esto supone una lectura un poco más lenta y también es más lento que una tercera opción que consiste en indicar explícitamente el esquema para los datos en el momento de la lectura, que es la opción recomendada si sabemos de antemano qué tipo va a tener cada columna. Si lo hiciésemos de esa manera, en caso de que no se pueda leer con ese esquema obtendríamos un error.

In [3]:
from pyspark.sql import functions as F

flightsDF = spark.read\
                 .option("header", "true")\
                 .option("inferSchema", "true")\
                 .csv("gs://ucmbucket/data/flights-jan-apr-2018.csv") # pon aquí la ruta en tu bucket

# Ensuciamos a propósito la variable ArrDelay para que pase a ser un string como suele pasar con frecuencia
flightsDF = flightsDF.withColumn("ArrDelay",\
                                 F.when(F.rand(seed = 123) < 0.1, "NA").otherwise(F.col("ArrDelay")))

flightsDF.printSchema()

root
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- FlightDate: timestamp (nullable = true)
 |-- Origin: string (nullable = true)
 |-- OriginCity: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- DestCity: string (nullable = true)
 |-- DepTime: integer (nullable = true)
 |-- DepDelay: double (nullable = true)
 |-- ArrTime: integer (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- Cancelled: double (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: double (nullable = true)
 |-- ActualElapsedTime: double (nullable = true)
 |-- AirTime: double (nullable = true)
 |-- Distance: double (nullable = true)
 |-- CarrierDelay: double (nullable = true)
 |-- WeatherDelay: double (nullable = true)
 |-- NASDelay: double (nullable = true)
 |-- SecurityDelay: double (nullable = true)
 |-- LateAircraftDelay: double (nullable = true)



Ahora tiene mejor pinta, aunque todavía hay algunas columnas cuyo tipo de dato sigue siendo string cuando la intuición nos dice que deberían ser enteros.

## Operaciones básicas con Data Frames

### Contamos el número de filas

Es una de las primeras cosas que nos preguntamos sobre un dataset: número de filas y columnas (cuántos ejemplos y cuántas variables tenemos para describirlos). Puesto que vamos a llevar a cabo varias transformaciones al DataFrame `flightsDF` a partir de este punto, vamos a usar `cache`() para que Spark lo mantenga en memoria en lugar de liberar la memoria ocupada tras cada acción.

In [6]:
# Extraemos los nombres de columna. Esto son solo metadatos de DataFrame, y están en el driver. No es necesaria ninguna
# operación sobre el cluster para recuperar la variable interna columns de cualquier DataFrame
print("Los datos tienen {0} columnas".format(len(flightsDF.columns)))

flightsDF.cache()        # Esta línea no hace cálculos, pero Spark anota que debe mantener este DF en memoria tras la primera vez que sea materializado
rows = flightsDF.count() # Esto es una acción que obligará a que flightsDF sea materializado. Para ello, habrá que llevar a cabo
                         # las transformaciones que lo generan en la celda anterior: read y withColumn, que están pendientes

print("Los datos tienen {0} filas".format(rows))

Los datos tienen 23 columnas
Los datos tienen 1182818 filas


### Seleccionar columnas por nombre

In [7]:
flightsDF.select("Month", "DayofMonth", "ArrTime")\
         .show() # los nombres son sensibles a mayúsculas

+-----+----------+-------+
|Month|DayofMonth|ArrTime|
+-----+----------+-------+
|    1|        14|   null|
|    1|         3|   1506|
|    1|         6|   1543|
|    1|         7|   1455|
|    1|         8|   1509|
|    1|         9|   1504|
|    1|        10|   1455|
|    1|        11|   1452|
|    1|        12|   1748|
|    1|        13|   1514|
|    1|        15|   1456|
|    1|        16|   1511|
|    1|        17|   1622|
|    1|        18|   1509|
|    1|        19|   1449|
|    1|        20|   1533|
|    1|        21|   1508|
|    1|        22|   1504|
|    1|        23|   1616|
|    1|        24|   1515|
+-----+----------+-------+
only showing top 20 rows



### Filtramos (retenemos) filas en base a los valores de una o varias columnas

La mayoría de las transformaciones están definidas en el paquete `pyspark.sql.functions`, por lo que es frecuente importar el paquete completo con un alias, como `F`, en lugar de importar cada función individual. A partir de ese momento usamos `F.` antes del nombre de cada función, para decirle a python dónde buscar esa función.

In [8]:
# La función col se utiliza para decir que nos estamos refiriendo a la columna cuyo nombre se pasa como argumento

flightsJanuary20 = flightsDF\
                      .where((F.col("DayofMonth") == 20) & (F.col("Month") == 1))\
                      .select("Month", "ArrTime") # encadenamos dos transformaciones: esto no desencadena ninguna operación

# Cúantos vuelos hay el 20 de enero de 2018
# La operación count() es una acción, así que obliga a materializar flightsJanuary20. Para ello es necesario ejecutar
# las transformaciones where() y select() que pusimos en esta celda, aplicadas a flightsDF. De hecho, si no hubiésemos
# cacheado flightsDF en las celdas anteriores, también habría que materializarlo otra vez, y para eso se leería de 
# nuevo el CSV desde HDFS
rowsJanuary20 = flightsJanuary20.count()

print("Hubo {0} vuelos el 20 de enero de 2018".format(rowsJanuary20))

# Esto es otra acción aplicada sobre el DataFrame flightsJanuary. Como flightsJanuary NO ha sido cacheado,
# entonces las operaciones "where" y "select" se necesitan ejecutar DE NUEVO para poder hacer el show()
flightsJanuary20.show(3)

Hubo 16176 vuelos el 20 de enero de 2018
+-----+-------+
|Month|ArrTime|
+-----+-------+
|    1|   1533|
|    1|   1734|
|    1|   2025|
+-----+-------+
only showing top 3 rows



**No olvidemos cachear el DataFrame cuando tengamos previsto hacer varias operaciones sobre él, o de lo contrario estaremos repitiendo muchas veces los cálculos previos que llevaron a ese DataFrame!!**

In [8]:
# También podemos indicar el filtrado como un string con un trozo de código SQL
# Recordemos que where y filter son exactamente equivalentes
flightsJanuary31 = flightsDF.filter("DayofMonth = 31 and Month = 1") # transformación filter: Spark no ejecuta nada

flightsJanuary31.count() # acción count: obliga a materializar flightsJanuary31 para lo cual se tiene que ejecutar filter

20519

<div class="alert alert-block alert-info">
<b>RECUERDA:</b> Spark hace todas estas operaciones de manera distribuida en el cluster, por tanto cada nodo del cluster está filtrando filas de entre aquellas que están presentes en ese nodo (más precisamente, en ese executor). Cada executor envía al driver el recuento de cuántas filas ha filtrado, y los resultados son agregados en el driver para mostrar el recuento total.
</div>

<div class="alert alert-block alert-success">
<b>TU TURNO</b>: muestra por pantalla el retraso en la llegada, el aeropuerto de origen y de destino de aquellos vuelos que tuvieron lugar en Domingo y con un retraso a la llegada mayor de 15 minutos. Muestra el esquema de dicho DataFrame resultante.

</div>

In [28]:
from pyspark.sql import functions as F

(flightsDF.select(F.col("ArrDelay"),F.col("Origin"),F.col("Dest"),F.col("DayOfWeek"))\
 .where((F.col("ArrDelay") > 15) & (F.col("DayOfWeek") == 7))\
 .show(10))

+--------+------+----+---------+
|ArrDelay|Origin|Dest|DayOfWeek|
+--------+------+----+---------+
|   21.00|   BOS| BUF|        7|
|   16.00|   BOS| JFK|        7|
|   22.00|   LGA| MSN|        7|
|  136.00|   BOS| JFK|        7|
|  119.00|   DTW| JFK|        7|
|  687.00|   DTW| MBS|        7|
|   90.00|   CVG| DTW|        7|
|   61.00|   GNV| ATL|        7|
|   48.00|   ATL| SHV|        7|
|   45.00|   SHV| ATL|        7|
+--------+------+----+---------+
only showing top 10 rows



### Seleccionar filas únicas

Para hacerse mejor idea de cómo es una variable categórica, es lógico querer cuántos valores distintos existen en nuestro dataset. Si consideramos todas las columnas, sería raro tener dos filas exactamente iguales en todos los valores, pero si seleccionamos solamente una o unas pocas columnas, podemos ver cuántas combinaciones distintas de valores de esas columnas se dan en el dataset.


In [29]:
distinctFlights = flightsDF.distinct()  # distinct es una transformación que devuelve el DF sin las filas repetidas
distinctFlightsCount = distinctFlights.count() # count es una acción y provoca que se ejecute la transformación distinct

print("Hay {0} filas distintas".format(distinctFlightsCount))

Hay 1172410 filas distintas


Si seleccionamos solo las columnas `Origin` y `Dest` y nos quedamos con las filas distintas (quitamos repetidos), entonces obtenemos un DataFrame con los posibles aeropuertos de origen y destino, es decir, aquellos trayectos que existen en un vuelo (pueden aparecer varias veces porque seguramente existirán muchos vuelos a lo largo de 2018 entre un origen y un destino)

<div class="alert alert-block alert-success">
    <b>TU TURNO</b>: ¿<b>cuántas</b> combinaciones de Origin y Dest existen?
</div>

In [35]:
#(flightsDF.select(F.col("ArrDelay"),F.col("Origin"),F.col("Dest"),F.col("DayOfWeek"))\
# .where((F.col("ArrDelay") > 15) & (F.col("DayOfWeek") == 7))\
# .show(10))

combinaciones = (flightsDF.select(F.col("Origin"),F.col("Dest")).distinct().count())

print("Hay {0} combinaciones distintas de Origin y Dest".format(combinaciones))

Hay 5520 combinaciones distintas de Origin y Dest


In [31]:
flightsDF.printSchema()

root
 |-- Month: string (nullable = true)
 |-- DayofMonth: string (nullable = true)
 |-- DayOfWeek: string (nullable = true)
 |-- FlightDate: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- OriginCity: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- DestCity: string (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- Cancelled: string (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- Distance: string (nullable = true)
 |-- CarrierDelay: string (nullable = true)
 |-- WeatherDelay: string (nullable = true)
 |-- NASDelay: string (nullable = true)
 |-- SecurityDelay: string (nullable = true)
 |-- LateAircraftDelay: string (nullable = true)



**Respuesta en la siguiente celda:**

In [36]:
countOriginDests = (flightsDF.select(F.col("Origin"),F.col("Dest")).distinct().count())

print("Hay {0} combinaciones de un aeropuerto de origen y uno de destino".format(countOriginDests))

Hay 5520 combinaciones de un aeropuerto de origen y uno de destino


Aunque tengamos 2.5 millones de filas, solo hay 5795 combinaciones distintas de un aeropuerto de origen y otro de destino.

<div class="alert alert-block alert-success">
    <b>TU TURNO</b>: ¿<b>cuántos</b> aeropuertos de origen existen?
</div>

In [37]:
distinctOrigins = (flightsDF.select(F.col("Origin")).distinct().count())

print("Hay {0} aeropuertos desde los que puede partir un vuelo".format(distinctOrigins))

Hay 353 aeropuertos desde los que puede partir un vuelo


<div class="alert alert-block alert-success">
<b>TU TURNO</b>: vamos a hacer esto para cada columna, en un bucle, para hacernos a la idea de cuántos valores hay. Esto solo tiene sentido en realidad para columnas categóricas y no para numéricas donde casi todos los valores serán distintos.
</div>

In [42]:
for i in range(len(flightsDF.columns)):
    distinctValues = (flightsDF.select(flightsDF.columns[i]).distinct().count())
    columnName = flightsDF.columns[i]
    print("Existen {0} valores distintos en la columna {1}".format(distinctValues, columnName))


Existen 2 valores distintos en la columna Month
Existen 31 valores distintos en la columna DayofMonth
Existen 7 valores distintos en la columna DayOfWeek
Existen 59 valores distintos en la columna FlightDate
Existen 353 valores distintos en la columna Origin
Existen 347 valores distintos en la columna OriginCity
Existen 353 valores distintos en la columna Dest
Existen 347 valores distintos en la columna DestCity
Existen 1431 valores distintos en la columna DepTime
Existen 1244 valores distintos en la columna DepDelay
Existen 1441 valores distintos en la columna ArrTime
Existen 1286 valores distintos en la columna ArrDelay
Existen 2 valores distintos en la columna Cancelled
Existen 5 valores distintos en la columna CancellationCode
Existen 2 valores distintos en la columna Diverted
Existen 677 valores distintos en la columna ActualElapsedTime
Existen 649 valores distintos en la columna AirTime
Existen 1425 valores distintos en la columna Distance
Existen 935 valores distintos en la colu

### Crear una nueva columna o reemplazar una existente por el resultado de operar con columnas existentes

Tenemos las distancias en millas. Vamos a convertirlas a km multiplicando las millas por 1.61. De nuevo, cada nodo del cluster hará esta operación localmente con los datos que se encuentran en ese nodo. El DataFrame resultante estará repartido en los mismos nodos. **No hay movimiento de datos ya que no se necesita ninguna información que no esté presente en ese nodo** para efectuar la operación.

In [43]:
# withColumn is a transformation returing a new DataFrame with one extra column appended on the right
flightsWithKm = flightsDF.withColumn("DistanceKm", F.col("Distance") * 1.61)

flightsWithKm.printSchema()

flightsWithKm.show(3)

root
 |-- Month: string (nullable = true)
 |-- DayofMonth: string (nullable = true)
 |-- DayOfWeek: string (nullable = true)
 |-- FlightDate: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- OriginCity: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- DestCity: string (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- Cancelled: string (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- Distance: string (nullable = true)
 |-- CarrierDelay: string (nullable = true)
 |-- WeatherDelay: string (nullable = true)
 |-- NASDelay: string (nullable = true)
 |-- SecurityDelay: string (nullable = true)
 |-- LateAircraftDelay: string (nullable = true)
 |-- DistanceKm: double (nullable = true

El día de la semana es una variable entera. Utilizar un entero o una variable categórica es una decisión que depende de qué tipo de modelo vayamos a ajustar. ¿Tiene sentido considerar días de la semana "más grandes" o "más pequeños", es decir, algo que se incrementa conforme "se incrementa" el día de la semana de 1 a 7? Aquí vamos a **reemplazar** la columna `DayOfWeek` que ya existía, por una versión categórica como strings. Utilizamos `withColumn` pasándole como primer argumento el nombre de una columna que ya existe, lo cual indica que queremos reemplazarla, en el mismo lugar que ocupaba.

In [44]:
flightsCategoricalDay = flightsDF.withColumn("DayOfWeek", F.when(F.col("DayOfWeek") == 1, "Monday")\
                                                           .when(F.col("DayOfWeek") == 2, "Tuesday")\
                                                           .when(F.col("DayOfWeek") == 3, "Wednesday")\
                                                           .when(F.col("DayOfWeek") == 4, "Thursday")\
                                                           .when(F.col("DayOfWeek") == 5, "Friday")\
                                                           .when(F.col("DayOfWeek") == 6, "Saturday")\
                                                           .otherwise("Sunday"))

flightsCategoricalDay.printSchema() # the column is still in the same position but has now string type

flightsCategoricalDay.select("DayOfWeek", "DepTime", "ArrTime").show()

root
 |-- Month: string (nullable = true)
 |-- DayofMonth: string (nullable = true)
 |-- DayOfWeek: string (nullable = false)
 |-- FlightDate: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- OriginCity: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- DestCity: string (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- Cancelled: string (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- Distance: string (nullable = true)
 |-- CarrierDelay: string (nullable = true)
 |-- WeatherDelay: string (nullable = true)
 |-- NASDelay: string (nullable = true)
 |-- SecurityDelay: string (nullable = true)
 |-- LateAircraftDelay: string (nullable = true)

+---------+-------+-------+
|DayOfWeek

<div class="alert alert-block alert-info">
    <b>CONSEJO:</b> El proceso de crear nuevas variables o <i>features</i> a partir de otras existentes, incluso incorporar variables de fuentes de datos externas (datos públicos) y de limpiar o reemplazar variables tras normalizarla se denomina genéricamente <i>feature engineering</i> (ingeniería de variables). A veces tiene mucho que ver con conocimientos específicos del dominio, aunque también con trucos estadísticos para normalizar siguiend métodos bien conocidos.
</div>

La función `when` es muy común para re-categorizar una variable o para crear nuevas variables categóricas a partir de condiciones complejas acerca de lo que les ocurre a los valores de otras columnas en esa misma fila.

<div class="alert alert-block alert-success">
<b>TU TURNO</b>: crea una variable categórica de tipo string con dos categorías indicando si el día de la semana es laborable o es fin de semana. Los valores deben ser "laborable" o "finde". Utiliza `withColumn` y `when`. Será laborable cuando DayOfWeek esté entre 1 y 5, y fin de semana cuando sea 6 o 7 (en resumen: "en otro caso" es fin de semana). 
</div>

In [55]:
flightsCategoricalLabor = flightsDF.withColumn("DayOfWeek", F.when((F.col("DayOfWeek") >= 1)|(F.col("DayOfWeek") <=5 ), "laborable")\
                                                           .otherwise("finde"))

flightsCategoricalLabor.printSchema() # the column is still in the same position but has now string type

flightsCategoricalLabor.select("DayOfWeek", "DepTime", "ArrTime").show()

root
 |-- Month: string (nullable = true)
 |-- DayofMonth: string (nullable = true)
 |-- DayOfWeek: string (nullable = false)
 |-- FlightDate: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- OriginCity: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- DestCity: string (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- Cancelled: string (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- Distance: string (nullable = true)
 |-- CarrierDelay: string (nullable = true)
 |-- WeatherDelay: string (nullable = true)
 |-- NASDelay: string (nullable = true)
 |-- SecurityDelay: string (nullable = true)
 |-- LateAircraftDelay: string (nullable = true)

+---------+-------+-------+
|DayOfWeek

### Crear y seleccionar columnas al vuelo

`withColumn` no es la única manera de crear columnas. Podemos usar `select` no solo para seleccionar columnas existentes sino también para crear nuevas columnas al vuelo y a la vez seleccionarlas. Vamos a seleccionar Origin y Dest, que ya existen, a la vez que seleccionamos una tercera columna que estamos creando al vuelo, con la transformación de la distancia a km. Le ponemos como nombre "DistanceKm" en ese momento que la estamos creando, mediante la función `alias`. Si no usamos `alias`, Spark le pondrá a la nueva columna un nombre por defecto que en este caso sería "1.6 * Distance". Se recomienda usar alias para dar nombre a las nuevas columnas.

In [56]:
flightsDF.selectExpr("Origin", "Dest", "1.6*Distance as DistanceKm").show()

+------+----+----------+
|Origin|Dest|DistanceKm|
+------+----+----------+
|   SYR| DTW|     598.4|
|   SYR| LGA|     316.8|
|   SYR| LGA|     316.8|
|   SYR| LGA|     316.8|
|   SYR| LGA|     316.8|
|   SYR| LGA|     316.8|
|   SYR| LGA|     316.8|
|   SYR| LGA|     316.8|
|   SYR| LGA|     316.8|
|   SYR| LGA|     316.8|
|   SYR| LGA|     316.8|
|   SYR| LGA|     316.8|
|   SYR| LGA|     316.8|
|   SYR| LGA|     316.8|
|   SYR| LGA|     316.8|
|   SYR| LGA|     316.8|
|   SYR| LGA|     316.8|
|   SYR| LGA|     316.8|
|   SYR| LGA|     316.8|
|   SYR| LGA|     316.8|
+------+----+----------+
only showing top 20 rows



In [57]:
flightsAirportsAndKm = flightsDF.select(F.col("Origin"),\
                                        F.col("Dest"),\
                                        (1.6 * F.col("Distance")).alias("DistanceKm"))

flightsAirportsAndKm.show(5)

+------+----+----------+
|Origin|Dest|DistanceKm|
+------+----+----------+
|   SYR| DTW|     598.4|
|   SYR| LGA|     316.8|
|   SYR| LGA|     316.8|
|   SYR| LGA|     316.8|
|   SYR| LGA|     316.8|
+------+----+----------+
only showing top 5 rows



<div class="alert alert-block alert-success">
<b>TU TURNO</b>: crear un DataFrame con tres columnas seleccionando "Origin", "OriginCity" y una nueva columna de tipo string creada concatenando Origin y OriginCity con un guión "-". Utilizar `withColumn` y dentro la función `concat_ws` (concatenar con separador) con sintaxis <a target="_blank" href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.concat_ws">F.concat_ws("-", columna1, columna2)</a> del paquete pyspark.sql.functions.
</div>

In [59]:
flightsCityConcat = flightsDF.withColumn("OriginConcat", F.concat_ws("-",F.col("Origin"),F.col("OriginCity")))\
                        .select(F.col("Origin"), F.col("OriginCity"), F.col("OriginConcat"))

flightsCityConcat.show()

+------+------------+----------------+
|Origin|  OriginCity|    OriginConcat|
+------+------------+----------------+
|   SYR|Syracuse, NY|SYR-Syracuse, NY|
|   SYR|Syracuse, NY|SYR-Syracuse, NY|
|   SYR|Syracuse, NY|SYR-Syracuse, NY|
|   SYR|Syracuse, NY|SYR-Syracuse, NY|
|   SYR|Syracuse, NY|SYR-Syracuse, NY|
|   SYR|Syracuse, NY|SYR-Syracuse, NY|
|   SYR|Syracuse, NY|SYR-Syracuse, NY|
|   SYR|Syracuse, NY|SYR-Syracuse, NY|
|   SYR|Syracuse, NY|SYR-Syracuse, NY|
|   SYR|Syracuse, NY|SYR-Syracuse, NY|
|   SYR|Syracuse, NY|SYR-Syracuse, NY|
|   SYR|Syracuse, NY|SYR-Syracuse, NY|
|   SYR|Syracuse, NY|SYR-Syracuse, NY|
|   SYR|Syracuse, NY|SYR-Syracuse, NY|
|   SYR|Syracuse, NY|SYR-Syracuse, NY|
|   SYR|Syracuse, NY|SYR-Syracuse, NY|
|   SYR|Syracuse, NY|SYR-Syracuse, NY|
|   SYR|Syracuse, NY|SYR-Syracuse, NY|
|   SYR|Syracuse, NY|SYR-Syracuse, NY|
|   SYR|Syracuse, NY|SYR-Syracuse, NY|
+------+------------+----------------+
only showing top 20 rows



### Convertir el tipo de dato de una columna

Con frecuencia, Spark no infiere correctamente el tipo de cada columna. 
* A veces ocurre que una columna numérica no se reconozca adecuadamente porque los datos tienen "NA" (un string) que quería significar data faltante en el dataset original. "NA" no es un string especial para Spark, por lo que simplemente reconoce que la columna tiene tanto enteros como strings, y el tipo más general que infiere es string para esa columna. 
* Pero no es el caso aquí porque no se da esto, y por eso hemos ensuciado al principio los datos a propósito. Vamos a limpiarlos y a convertir la columna `ArrDelay` a DoubleType como debe ser.

In [60]:
naCount = flightsDF.where("ArrDelay = 'NA'").count()

# Esto es sintaxis SQL, pero también podríamos haberla llamado como .where(F.col("ArrDelay") == "NA"). Ambas son equivalentes.

print("Hay ", naCount, "filas con NA en ArrDelay")

Hay  0 filas con NA en ArrDelay


In [61]:
flightsDF.where(F.col("ArrDelay") != "NA").count()

1149702

In [62]:
from pyspark.sql.types import DoubleType

flightsDF = flightsDF.where((F.col("ArrDelay") != "NA"))\
                     .withColumn("ArrDelay", F.col("ArrDelay").cast(DoubleType()))

### Ordenación respecto a una columna

Es una transformación que nos devuelve otro DataFrame ordenado según la(s) columna(s) indicada(s), sea de forma ascendente o descendente. El DataFrame original no se modifica (al igual que ocurre en cualquier transformación). Se puede ordenar en base a una columna numérica (lo más frecuente) o también categórica (los strings se ordenan alfabéticamente). Para ordenar sí es necesario enviar información de unos nodos a otros puesto que no sabemos si existen o no datos mayores o menos que el valor que tenemos en el nodo (o más precisamente, en el worker).

In [63]:
# Ordenamos los vuelos según ArrDelay
sortedDF = flightsDF.orderBy("ArrDelay")  # equivalente: flightsDF.orderBy(F.col("ArrDelay"))

sortedDescDF = flightsDF.orderBy("ArrDelay", ascending = False)
sortedDescDF.select("ArrDelay", "Origin", "Dest").show(10)

+--------+------+----+
|ArrDelay|Origin|Dest|
+--------+------+----+
|  2475.0|   HNL| PPG|
|  2454.0|   PPG| HNL|
|  2023.0|   EGE| JFK|
|  1778.0|   ORF| DFW|
|  1757.0|   SMF| DFW|
|  1717.0|   HNL| JFK|
|  1704.0|   EGE| DFW|
|  1648.0|   SLC| DFW|
|  1558.0|   DAY| DFW|
|  1543.0|   LBB| DFW|
+--------+------+----+
only showing top 10 rows



### Funciones de agregación sobre el DataFrame completo

Spark tiene implementaciones distribuidas y paralelas de funciones de agregación frecuentes como la media, min, max y desviación típica entre otras. Todas estas funciones reciben como argumento el objeto columna sobre el que la función debe aplicarse. Lo más habitual es aplicarlas para agregar por grupos, pero esto lo veremos en el segundo notebook.

Vamos a seleccionar vuelos con `ArrDelay` mayor de 15 minutos, y con ellos vamos a crear columnas con el min, max, media y desviación típica de la columna `ArrDelay`. Crearemos y seleccionaremos dichas columnas al vuelo, como vimos en la sección anterior.

In [64]:
flightsDF.printSchema()

root
 |-- Month: string (nullable = true)
 |-- DayofMonth: string (nullable = true)
 |-- DayOfWeek: string (nullable = true)
 |-- FlightDate: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- OriginCity: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- DestCity: string (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- ArrDelay: double (nullable = true)
 |-- Cancelled: string (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- Distance: string (nullable = true)
 |-- CarrierDelay: string (nullable = true)
 |-- WeatherDelay: string (nullable = true)
 |-- NASDelay: string (nullable = true)
 |-- SecurityDelay: string (nullable = true)
 |-- LateAircraftDelay: string (nullable = true)



In [65]:
flightsDF.count()

1149702

In [66]:
# First we select those flights with at least 16 minutes of delay and then compute the aggregations
flightsDF.where(F.col("ArrDelay") > 15)\
         .select(F.mean("ArrDelay").alias("MeanArrDelay"),\
                 F.min("ArrDelay").alias("MinArrDelay"),\
                 F.max("ArrDelay").alias("MaxArrDelay"),
                 F.stddev("ArrDelay").alias("StddevArrDelay")).show()

+-----------------+-----------+-----------+--------------+
|     MeanArrDelay|MinArrDelay|MaxArrDelay|StddevArrDelay|
+-----------------+-----------+-----------+--------------+
|67.70143687510016|       16.0|     2475.0|87.68094388409|
+-----------------+-----------+-----------+--------------+



Existe una función de Spark que hace casi todo esto por nosotros, llamada `summary`. Lo hace para cada columna numérica que encuentre en el dataset. Es también una transformación que devuelve un nuevo DataFrame con las métricas de resumen, sin modificar el DataFrame original.

In [67]:
summariesDF = flightsDF.summary()
summariesDF.show()

+-------+------------------+------------------+------------------+----------+-------+------------+-------+------------+------------------+----------------+------------------+------------------+---------+----------------+--------+------------------+------------------+-----------------+-----------------+-----------------+------------------+-------------------+------------------+
|summary|             Month|        DayofMonth|         DayOfWeek|FlightDate| Origin|  OriginCity|   Dest|    DestCity|           DepTime|        DepDelay|           ArrTime|          ArrDelay|Cancelled|CancellationCode|Diverted| ActualElapsedTime|           AirTime|         Distance|     CarrierDelay|     WeatherDelay|          NASDelay|      SecurityDelay| LateAircraftDelay|
+-------+------------------+------------------+------------------+----------+-------+------------+-------+------------+------------------+----------------+------------------+------------------+---------+----------------+--------+-----------

### Conversión a Pandas

En general es complicado leer los resultados que muestra Spark. En ocasiones interesa convertirlos a un dataframe de Pandas, aunque esto implica traer todas las filas al driver, lo cual **debe hacerse con mucho cuidado y solo en casos en los que estemos seguros de que el DataFrame de Spark es pequeño y no desbordará la memoria del driver**.

<div class="alert alert-block alert-info">
<p><b>CONSEJO</b>: el concepto de dataframe como tabla con filas y columnas existe en muchos lenguajes de programación (pandas de Python, dataframes de R, DataFrames de Spark). Sin embargo, Spark maneja DataFrames que están físicamente distribuidos en las memorias RAM de las máquinas del cluster, lo cual no tiene nada que ver con lo que hace la biblioteca Pandas o R que se ejecutan en una sola máquina. Convertir un DataFrame de Spark en un dataframe de Pandas implica llevar todas las filas al driver, lo cual podría resultar en una excepción Out-of-Memory si el contenido del DataFrame es más grande que la memoria RAM de la máquina en la que se está ejecutando el programa driver. En este caso y en la mayoría de casos, se suele utilizar para mostrar resúmenes o agregados ya calculados previamente, y que sabemos que ocupan poco, con lo que no existe riesgo.</p>

<p>Esta operación también es muy frecuente cuando queremos representar gráficamente el contenido de un DataFrame de Spark. No existen funciones gráficas en Spark, por lo que tenemos que convertirlo en un dataframe de Pandas y utilizar las funciones gráficas de Python habituales (matplotlib, Seaborn o incluso la propia biblioteca Pandas) para mostrar lo que necesitemos.
</p>
</div>

In [68]:
flightsPd = flightsDF.summary().toPandas()

In [69]:
flightsPd

,summary,Month,DayofMonth,DayOfWeek,FlightDate,Origin,OriginCity,Dest,DestCity,DepTime,...,CancellationCode,Diverted,ActualElapsedTime,AirTime,Distance,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,count,1149702,1149702,1149702,1149702,1149702,1149702,1149702,1149702,1149702,...,0,1149702,1149701,1148131,1149701,213274,213274,213274,213274,213274
1,mean,1.4773802254845168,15.461339547117428,3.815941870154179,None,None,None,None,None,1333.3615110698252,...,None,0.0,133.47236629349717,108.25110984722127,762.7137873238346,20.57006948807637,4.33073417294185,14.595117079437719,0.08588013541266165,26.29449440625674
2,stddev,0.4994883009677767,8.620475654572719,1.9900419945103371,None,None,None,None,None,493.2410880251395,...,None,0.0,72.09121000559432,70.279984018462,583.0594452064626,63.43487059862232,34.76113501722374,32.57310521074373,2.791568097809187,52.657823691375306
3,min,1,1,1,2018-01-01,ABE,"Aberdeen, SD",ABE,"Aberdeen, SD",0001,...,None,0.00,-1228.00,-1244.00,100.00,0.00,0.00,0.00,0.00,0.00
4,25%,1.0,8.0,2.0,None,None,None,None,None,923.0,...,None,0.0,82.0,57.0,337.0,0.0,0.0,0.0,0.0,0.0
5,50%,1.0,16.0,4.0,None,None,None,None,None,1329.0,...,None,0.0,115.0,89.0,601.0,0.0,0.0,2.0,0.0,2.0
6,75%,2.0,23.0,5.0,None,None,None,None,None,1737.0,...,None,0.0,164.0,138.0,1005.0,17.0,0.0,19.0,0.0,32.0
7,max,2,9,7,2018-02-28,YUM,"Yuma, AZ",YUM,"Yuma, AZ",2400,...,None,0.00,99.00,99.00,999.00,998.00,994.00,99.00,99.00,99.00
